#### Fargo2radmc3d runner

Runs fargo2radmc3d code


In [4]:
# pip install dotted-dict

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import section

import numpy as np
import sys
import os
import re
import pandas as pd
import shutil
import para
from dotted_dict import DottedDict
from scipy.stats import qmc
from scipy.interpolate import UnivariateSpline


In [6]:
p = DottedDict()
p.y = 56

x = 'dir'
val= 'hello'
p[x] = val
print(p)

a = int(float(str(1e10)))

DottedDict({'y': 56, 'dir': 'hello'})


In [7]:
def radmc3d_LHC_para_generator(params={}, number_of_samples=5, outfile=None):

    num = len(params)

    lower_bounds = np.array([value[0] for value in params.values()])
    upper_bounds = np.array([value[1] for value in params.values()])

    sampler = qmc.LatinHypercube(d=num, seed=7)
    sample = sampler.random(n=number_of_samples)
    sample_scaled = qmc.scale(sample, lower_bounds, upper_bounds)

    # find out if any parameter needs to be an integer
    dtype_list = [value[2] for value in params.values()]
    scale = [value[3] for value in params.values()]

    indices = [index for index, element in enumerate(dtype_list) if element == 'int']
    for i in indices:
        sample_scaled[:,i] = np.round(sample_scaled[:,i]).astype('int')

    indices = [index for index, scale_type in enumerate(scale) if scale_type == 'log10']
    for i in indices:
        sample_scaled[:,i] = 10**(sample_scaled[:,i])  


    # Create a dtype for structured array
    dtype = np.dtype([(header, dtype) for header,dtype in zip(params.keys(), dtype_list)])

    # Create structured array
    structured_samples_cube = np.empty(sample_scaled.shape[0], dtype=dtype)
    for i, header in enumerate(params.keys()):
        structured_samples_cube[header] = sample_scaled[:, i]

    # Define format specifiers based on data type
    format_specifiers = {
        'int': '%d',
        'float': '%.3e'  # Using scientific notation with 3 decimal places
    }

    # Format the data for saving
    formatted_samples_cube = []
    for row in structured_samples_cube:
        formatted_row = []
        for header, dt in zip(params.keys(), dtype_list):
            formatted_row.append(format_specifiers[dt] % row[header])
        formatted_samples_cube.append(formatted_row)

    formatted_samples_cube = [dtype_list] + formatted_samples_cube
    
    # print(formatted_samples_cube)
    
    if outfile != None:
        if os.path.isfile(outfile):
            print("Old data file detected, deleted, making new")
            os.remove(outfile)
            np.savetxt(fname=outfile, X=formatted_samples_cube, delimiter='\t', fmt='%s', 
                    header='\t'.join(params.keys()), comments='')
        else:
            np.savetxt(fname=outfile, X=formatted_samples_cube, delimiter='\t', fmt='%s', 
                    header='\t'.join(params.keys()), comments='')



# radmc3d_LHC_para_generator(params= parameters_dict,number_of_samples=1000,
#                            outfile = '/home/mihir/radmc3d_fargo/fargo2radmc3d_copy/LHC_radmc.dat')

In [2]:
def par_file_reader(default_par_file_path):
    """
    Reads default parameter file
    Returns a dictionary (parameter: value)
    """


    regex = re.compile(",")

    for line in lines_params:
        try:
            line.split()[0][0]  # check if blank line (GWF)
        except:
            continue
        if line.split()[0][0] == "#":  # check if line starts with a # (comment)
            continue
        else:
            name, value = line.split()[0:2]
        try:
            float(value)  # first trying with float
            if (
                name == "dustfluids" and value == "1"
            ):  # taking care of case when only one dust fluid is used
                value = int(1)
        except ValueError:  # if it is not float
            try:
                int(value)  # we try with integer
            except ValueError:  # if it is not integer nor float
                # we try with array with several values separated by a comma (,)
                if regex.search(value) != None:
                    if name == "dustfluids":
                        value = value#[int(x) for x in value.split(",")]
                else:
                    value = str(value)
                        #'"' + value + '"'
                    #)  # if none of the above tests work, we know value is a string


    return parameters


def dust_size_calc(inv_stokes, initial_density, grain_internal_density):
    """
    Calculates dust grain size using the input parameters and returns dust size in cm 
    """
    
    # Converting initial surface density to physical units gm/cm2
    # Sigma0 = Sigma_0*M_star/(R_0)^2 in fargo3d, We have set R0 = 5.2 AU, M_starr = M_sun

    initial_gas_dens = (initial_density * 1.989e33) / (5.2 * 1.496e13) ** 2.0

    dust_size = (2.0 * initial_gas_dens) / (
        inv_stokes * np.pi * grain_internal_density
    )  # dust size in cm

    return dust_size / 100.0  # dust size in meters


def fargo3d2radmc_runner(
                            default_parameter_file="/home/mihir/radmc3d_fargo/fargo_radmc3d_defaults/params.dat",
                            fargo3d_output_path="/home/mihir/radmc3d_fargo/fargo_1_dust_multi_planet_11",
                            fargo2radmc3d_path = "/home/mihir/radmc3d_fargo/fargo2radmc3d_copy",
                            radmc3d_parameters_file = "/home/mihir/radmc3d_fargo/fargo2radmc3d_copy/LHC_radmc.csv",
                            radmc_output_dir_path = None,
                            output_number="8",
                            dust_internal_rho=1.3,
                            FARGO3d_setup_required=True,
                            recalculate_radmc3d = "Yes",
                            recalc_opac = "Yes",
                            Number_of_cores = 2,
                            beamsize = 0.015,
                            recalc_rawfits = "Yes",
                            recalc_convolved_fluxmap = "Yes",
                            number_of_pixel = 256
                        ):
    """ """

    # Set up FARGO3d outputs
    if FARGO3d_setup_required:
        # Read FARGO3d variables.par file
        fargo_par_file = os.path.join(fargo3d_output_path, "variables.par")
        if os.path.exists(fargo_par_file):
            fargo3d_variables = dict(
                np.loadtxt(fargo_par_file, delimiter="\t",
                           usecols=(0, 1), dtype=str)
            )

            # Check if key "DUSTINTERNALRHO exist in the dictionary"
            if "DUSTINTERNALRHO" in fargo3d_variables.keys():
                # If it exists, for safe keeping change the corresponding value
                print(
                    f"""DUSTINTERNALRHO = {fargo3d_variables['DUSTINTERNALRHO']} already exist in FARGO3d variables.par changing the value to {dust_internal_rho} as specified for radiative transfer calculation."""
                )
                fargo3d_variables["DUSTINTERNALRHO"] = dust_internal_rho
            else:
                # If it doesn't exist, add the key: value pair in dict, and store it at the end of the file
                fargo3d_variables["DUSTINTERNALRHO"] = dust_internal_rho
                with open(fargo_par_file, "a") as f:
                    f.write(f"DUSTINTERNALRHO\t{dust_internal_rho}\n")
                print(
                    f"Added new parameter DUSTINTERNALRHO = {dust_internal_rho} in variables.par"
                )
        else:
            print("FARGO3d output variable.par file doesn't exist")
            return None



        #


    system_name = fargo3d_output_path.split("/")[-1]
    print(system_name)
    system_id = int(system_name.split("_")[-1])
    
    # Read the fargo2radmc3d default parameter file.
    radmc_parms = par_file_reader(default_parameter_file)

    radmc_parms['dir'] = fargo3d_output_path
    radmc_parms['on'] = output_number
    radmc_parms['recalc_radmc'] = recalculate_radmc3d
    radmc_parms['recalc_opac'] = recalc_opac
    radmc_parms['nbcores'] = Number_of_cores
    radmc_parms['recalc_rawfits'] = recalc_rawfits
    radmc_parms['recalc_fluxmap'] = recalc_convolved_fluxmap
    radmc_parms['bmaj'] = beamsize
    radmc_parms['bmin'] = beamsize

    # Sample temperature


    radmc_param_df = pd.read_csv(radmc3d_parameters_file, header=0, skiprows=[1], sep="\t")
    
    teff = radmc_param_df['Teff'].values[system_id]
    distance = radmc_param_df['distance'].values[system_id]
    inclination = radmc_param_df["inclination"].values[system_id]
    position_angle = radmc_param_df["position_angle"].values[system_id]

    radmc_parms['teff'] = np.round(teff)
    radmc_parms['distance'] = np.round(distance)
    radmc_parms['inclination'] = np.round(inclination, 1)
    radmc_parms['posangle'] = np.round(position_angle)
    radmc_parms['nbpixels'] = int(number_of_pixel)


    print(radmc_parms)


    for i in range(3):
        print("##################")
    print("RADMC3d run completed")
    for i in range(3):
        print("##################")

    if radmc_output_dir_path == None:
        radmc_output_dir = fargo3d_output_path.split(sep = "/")[-1] + "_RT_opt"
        radmc_output_dir_path = os.path.join(fargo2radmc3d_path, radmc_output_dir)
    print(f"Shifting outputs to {radmc_output_dir_path}")

    if not os.path.exists(radmc_output_dir_path):
        os.makedirs(radmc_output_dir_path)
    
    files = os.listdir(fargo2radmc3d_path)

    output_files_extentions = {".inp", ".dat", ".binp",".fits",".bdat",".out",".pdf"}

    for file in files:
        _, extension = os.path.splitext(file)
        if extension in output_files_extentions:
            source_file_path = os.path.join(fargo2radmc3d_path, file)
            destination_file_path = os.path.join(radmc_output_dir_path, file)

            try:
                # Move the file to the destination directory
                shutil.move(source_file_path, destination_file_path)
                print(f"Moved {file} to {radmc_output_dir_path}")
            except Exception as e:
                print(f"Error moving {file}: {str(e)}")
    

    return

In [6]:

# np.random.seed(14187941)

fargo3d2radmc_runner(
                        default_parameter_file="/home/mihir/radmc3d_fargo/fargo_radmc3d_defaults/params.dat",
                        fargo3d_output_path="/home/mihir/radmc3d_fargo/fargo_1_dust_multi_planet_992",
                        fargo2radmc3d_path = "/home/mihir/radmc3d_fargo/fargo2radmc3d_copy",
                        radmc3d_parameters_file = "/home/mihir/radmc3d_fargo/fargo2radmc3d_copy/LHC_radmc.csv",
                        radmc_output_dir_path = None,
                        output_number="50",
                        dust_internal_rho=1.3,
                        FARGO3d_setup_required=True,
                        recalculate_radmc3d = "Yes",
                        recalc_opac = "Yes",
                        Number_of_cores = 2,
                        beamsize = 0.015,
                        recalc_rawfits = "Yes",
                        recalc_convolved_fluxmap = "Yes",
                        number_of_pixel = 256
                    )

DUSTINTERNALRHO = 1.3 already exist in FARGO3d variables.par changing the value to 1.3 as specified for radiative transfer calculation.
fargo_1_dust_multi_planet_992
Opening /home/mihir/radmc3d_fargo/fargo_radmc3d_defaults/params.dat
['/home/mihir/radmc3d_fargo/fargo_1_dust_multi_planet_992', '50', 'Yes', '7.5e11', '2e30', 'Yes', 'dust', 'No', 'Yes', 'Yes', 'Yes', 'No', 'No', 'No', '3e-5', 'No', 'No', 'No', '50', '5.0', '256', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', '1.3', '2', '0', '1e7', '1e7', 1, '1e-5', '1e-2', '3.5', '1e-2', '3', 'G', 'mix_2species_60silicates_40carbons', 'No', '/home/mihir/radmc3d_fargo/opac', 'No', '0.0', '0.0', 'Qphi', 'No', 'Yes', 'No', '1', 'co', '2', '1e-4', '0.0', '10.0', '0', '81', '0.0', 'Yes', 'Yes', 'No', '140.0', '15.7', 'No', '0.0', '180', '1.6', '9500.0', '0.0005', '0.0005', '76.4', 'No', '0.4', 'inferno', 'No', '#', '#']
{'dir': '/home/mihir/radmc3d_fargo/fargo_1_dust_multi_planet_992', 'on': '50', 'override_units': 'Yes', 'new_unit_length': 

Total dust mass [g] =  8.071611044038493e+29
Total dust mass [Mgas] =  0.009943463262833468
Total dust mass [Mstar] =  0.0004035805522019246
Maximum dust surface density [in g/cm^2] is  12.11489214579795
--------- computing dust mass surface density ----------
--------- computing dust mass volume density ----------
max(rho_dustcube) [g cm-3] for colatitude index j =  0  =  1.0390984062648774e-17
max(rho_dustcube) [g cm-3] for colatitude index j =  1  =  2.703179932669149e-17
max(rho_dustcube) [g cm-3] for colatitude index j =  2  =  6.71459399464315e-17
max(rho_dustcube) [g cm-3] for colatitude index j =  3  =  1.5932596458372755e-16
max(rho_dustcube) [g cm-3] for colatitude index j =  4  =  3.6130968107959333e-16
max(rho_dustcube) [g cm-3] for colatitude index j =  5  =  7.83450143945975e-16
max(rho_dustcube) [g cm-3] for colatitude index j =  6  =  1.6251821393799426e-15
max(rho_dustcube) [g cm-3] for colatitude index j =  7  =  3.2268516701494016e-15
max(rho_dustcube) [g cm-3] for c

<Figure size 576x576 with 0 Axes>

In [ ]:
def rad_calc(T_eff, EV_data_file_path):
    """
    Calculates radius of a one solar mass star from its temperature
    """

    T_eff,radius = np.loadtxt(EV_data_file_path, usecols=(2,5),skiprows=1,unpack=True)

    

    



In [ ]:
print(
    dust_size_calc(
        inv_stokes=73.34, initial_density=1.839e-4, grain_internal_density=1.3
    )
)
print(
    f"Max dust size = {dust_size_calc(inv_stokes=10.,initial_density=5.0e-4,grain_internal_density=1.3): .3e} meters"
)
print(
    f"Min dust size = {dust_size_calc(inv_stokes=1000.,initial_density=5.0e-5,grain_internal_density=1.3): .3e} meters"
)